# Combinatorics with SageMath: Databases

## Nadia Lafrenière
### Dartmouth College (🇺🇸) $\xrightarrow{🚌}$ Concordia University (🇨🇦) 

## EAUMP-ICTP School on Enumerative Combinatorics
### Arusha, Tanzania, July 26, 2023

Acknowledgements: 
- Duncan Levear, from Boston College, made a part of this worksheet (in 2019), and gave me permission to use it
- Steven Melczer's [worksheet](https://melczer.ca/files/TextbookCode/Chapter2/Example2-4-VirahankaFibonacci.html) on the Fibonacci numbers

# A motivating example

Professor Balázs gave you this exercise:
> a. In how many ways can I pay 10 shillings if I am only allowed 1 and 2-shilling coins? How about 11 shillings? 20 shillings?

> b. In how many ways can I pay n shillings, if I am only allowed 1 and 2-shilling coins? Can you find a general formula for the number of ways? Can you prove your formula?

I would like you to say how you would do it with combinatorial structures, as we saw them yesterday.


We want to partition the integer 10 in any way with only parts 1 and 2!

In [1]:
Partitions(10, max_part=2).cardinality()

6

We could also check how many that is for all the numbers between 10 and 20.

In [2]:
for n in range(10, 21):
    print(n, Partitions(n, max_part=2).cardinality())

10 6
11 6
12 7
13 7
14 8
15 8
16 9
17 9
18 10
19 10
20 11


It seems that the sequence is [1,2,2,3,3,4,4,,..].

If you did that exercise the way Professor Balász suggested you, you would get the generating function
$$f(x) = \frac{-x^2 + x + 1}{x^3 - x^2 - x + 1}.$$

Can we expand it as to get the answer for 10, 11 and 20 shillings all at once?

In [3]:
f(x) = (-x^2 + x + 1)/(x^3 - x^2 - x + 1)
f.series(x,21)

x |--> 1 + 2*x + 2*x^2 + 3*x^3 + 3*x^4 + 4*x^5 + 4*x^6 + 5*x^7 + 5*x^8 + 6*x^9 + 6*x^10 + 7*x^11 + 7*x^12 + 8*x^13 + 8*x^14 + 9*x^15 + 9*x^16 + 10*x^17 + 10*x^18 + 11*x^19 + 11*x^20 + Order(x^21)

**Generating functions** might seem like a complicated technique for this, in comparison with the partition approach. The power of generating function comes from the fact that one could get much more information about them. For example, we could simultaneously keep track of the number of coins needed for each way, and the amount. Then, the coefficients of the *bivariate generating functions* are the number of ways to pay a fixed amount using a fixed number of coins.

## Bivariate generating functions


### [Lecture at the board: bivariate generating functions]

### Motivating example: the chicken nuggets problem

McDonald's, a famous chain of fast food in America, used to sell chicken nuggets in packs of 6, 9 or 20. What is the largest number of nuggets that one cannot order?

In [4]:
S = (1/(1-x^6-x^9-x^20)).series(x,100); S

1 + 1*x^6 + 1*x^9 + 1*x^12 + 2*x^15 + 2*x^18 + 1*x^20 + 3*x^21 + 4*x^24 + 2*x^26 + 5*x^27 + 2*x^29 + 7*x^30 + 3*x^32 + 9*x^33 + 6*x^35 + 12*x^36 + 7*x^38 + 16*x^39 + 1*x^40 + 12*x^41 + 21*x^42 + 17*x^44 + 28*x^45 + 3*x^46 + 24*x^47 + 37*x^48 + 3*x^49 + 36*x^50 + 49*x^51 + 6*x^52 + 50*x^53 + 65*x^54 + 12*x^55 + 72*x^56 + 86*x^57 + 16*x^58 + 102*x^59 + 115*x^60 + 30*x^61 + 143*x^62 + 151*x^63 + 45*x^64 + 202*x^65 + 204*x^66 + 70*x^67 + 282*x^68 + 269*x^69 + 111*x^70 + 394*x^71 + 361*x^72 + 165*x^73 + 549*x^74 + 485*x^75 + 253*x^76 + 762*x^77 + 646*x^78 + 378*x^79 + 1058*x^80 + 876*x^81 + 561*x^82 + 1462*x^83 + 1176*x^84 + 833*x^85 + 2024*x^86 + 1592*x^87 + 1221*x^88 + 2789*x^89 + 2163*x^90 + 1788*x^91 + 3847*x^92 + 2933*x^93 + 2603*x^94 + 5298*x^95 + 4008*x^96 + 3771*x^97 + 7282*x^98 + 5474*x^99 + Order(x^100)

I would like to know what powers do not appear in this expansion, without having to inspect it manually.

In [5]:
nonuggets = [t[1] for t in S.coefficients() if t[0]==0]
print("You cannot make packs of " + str(nonuggets))

You cannot make packs of [1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 16, 17, 19, 22, 23, 25, 28, 31, 34, 37, 43]


#### Two more combinatorial questions
> How many ways are there to make 60 chicken nuggets?

> How many of those ways use $7$ packs?

In [6]:
# How many ways are there to make 60 chicken nuggets?


In [7]:
# How many ways are there to make 60 chicken nuggets?
S.series(x, 61).coefficients()[60]

[115, 60]

There are 115 ways to get 60 chicken nuggets.

To get the number of ways that use 7 packs, I need a variable that keep tracks of the number of packs. Let $y$ be this variable.

Then, the generating function needs to be something like
$$\sum_{n\geq 0}\sum_{k\geq 0}\sum_{P\text{ partition of $n$ nuggets into $k$ packs of 6, 9 or 20}} y^k x^n.$$

This is the same as 
$$\sum_{k\geq 0} \left(y(x^6+x^9+x^{20})\right)^k = \frac{1}{1-y(x^6+x^9+x^{20})}.$$

In [8]:
# In class

In [9]:
var('y')
S = (1/(1-y*x^6 - y*x^9 - y*x^20))
S.series(x,100).coefficients()[60][0].expand()

y^10 + 36*y^9 + 70*y^8 + 7*y^7 + y^3

So there are 7 ways to make 7 packs of nuggets, for a total of 60.

What does the above give if you don't place "expand"?

In [10]:
var('y')
(1/(1-y*x^6 - y*x^9 - y*x^20)).series(x,100).coefficients()[60][0]

y^3 + ((((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + (3*y^4 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (y^3 + y^2)*y)*y + (2*y^3 + (y^3 + y^2)*y)*y)*y)*y)*y + (((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + (3*y^4 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (y^3 + y^2)*y)*y + (2*y^3 + (y^3 + y^2)*y)*y)*y)*y)*y + ((((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + (3*y^4 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (y^3 + y^2)*y)*y + (2*y^3 + (y^3 + y^2)*y)*y)*y)*y)*y + (((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (y^3 + y^2)*y)*y + (2*y^3 + (y^3 + y^2)*y)*y)*y)*y + (((3*y^4 + (2*y^3 + (y^3 + y^2)*y)*y)*y + (3*y^4 + (y^3 + y^2)*y)*y)*y + ((3*y^4 + (y^3 + y^2)*y)*y + (2*y^3 + (y^3 + y^2)*y)*y)*y)*y)*y)*y

We can also add variables that keep track of all the types of packs (either *six* ($s$), *nine* ($n$) or *twenty* ($t$).

In [11]:
# In class

In [12]:
var('s,n,t')
(1/(1-s*x^6 - n*x^9 - t*x^20)).series(x,100).coefficients()[60][0].expand()

s^10 + 36*n^2*s^7 + 70*n^4*s^4 + 7*n^6*s + t^3

### Multivariate generating functions: Takeaway

Actually, there is no limit on the number of variables one can define, but it might be hard to keep track of all the information we are interested in.

# Getting the generating function

I think you have seen two types of recursions to play with generating functions: a simpler one is linear recursion, and another one uses the symbolic method.

## Catalan generating function using the symbolic method

In Hans's lecture on Monday, he did a long computation to get the Catalan numbers from the symbolic method.

Let $F$ be the generating function for Dyck paths, and let $x$ track the number of pairs of steps (one up, one down). Using the first return, we saw that the generating function $$C = \sum_{P \text{ Dyck Paths}} x^{\# \text{up-steps}(P)}$$ satisifes the equation $C = 1 + xC^2$. 

> Pass this generating function to Sage.

In [13]:
# In class

In [14]:
C, x = var('C,x')
sys = [ C == x + C*C ]

Now, let's solve this generating function. As we saw in Hans's lecture, there are two answers, but only one has nice coefficients.

In [15]:
# In class

In [16]:
sol = solve(sys, C); sol

[C == -1/2*sqrt(-4*x + 1) + 1/2, C == 1/2*sqrt(-4*x + 1) + 1/2]

There are two solutions, only one of which is right. Expand them to figure out how to do that.

In [17]:
# In class

In [18]:
C1(x) = sol[0].right_hand_side(); C2(x) = sol[1].right_hand_side()

In [19]:
# In class

In [20]:
# In class

In [21]:
C1.series(x)

x |--> 1*x + 1*x^2 + 2*x^3 + 5*x^4 + 14*x^5 + 42*x^6 + 132*x^7 + 429*x^8 + 1430*x^9 + 4862*x^10 + 16796*x^11 + 58786*x^12 + 208012*x^13 + 742900*x^14 + 2674440*x^15 + 9694845*x^16 + 35357670*x^17 + 129644790*x^18 + 477638700*x^19 + Order(x^20)

In [22]:
C2.series(x)

x |--> 1 + (-1)*x + (-1)*x^2 + (-2)*x^3 + (-5)*x^4 + (-14)*x^5 + (-42)*x^6 + (-132)*x^7 + (-429)*x^8 + (-1430)*x^9 + (-4862)*x^10 + (-16796)*x^11 + (-58786)*x^12 + (-208012)*x^13 + (-742900)*x^14 + (-2674440)*x^15 + (-9694845)*x^16 + (-35357670)*x^17 + (-129644790)*x^18 + (-477638700)*x^19 + Order(x^20)

The latter has a lot of negative numbers, so we should avoid it. Then, the Dyck Paths would be counted by the generating function 'C1' here,

In [23]:
C1.series(x, 10)

x |--> 1*x + 1*x^2 + 2*x^3 + 5*x^4 + 14*x^5 + 42*x^6 + 132*x^7 + 429*x^8 + 1430*x^9 + Order(x^10)

In [24]:
oeis([1, 1, 2, 5, 14, 42, 132, 429, 1430])  # Requires internet.

0: A000108: Catalan numbers: C(n) = binomial(2n,n)/(n+1) = (2n)!/(n!(n+1)!).
1: A120588: G.f. is 1 + x*c(x), where c(x) is the g.f. of the Catalan numbers (A000108).
2: A211216: Expansion of (1-8*x+21*x^2-20*x^3+5*x^4)/(1-9*x+28*x^2-35*x^3+15*x^4-x^5).

We can inspect the OEIS entry A000108 to see that what we got is indeed the generating function for the Catalan numbers. 

In [25]:
oeis('A000108').formulas()[0:4]

0: a(n) = binomial(2*n, n)/(n+1) = (2*n)!/(n!*(n+1)!) = A000984(n)/(n+1).
1: Recurrence: a(n) = 2*(2*n-1)*a(n-1)/(n+1) with a(0) = 1.
2: Recurrence: a(n) = Sum_{k=0..n-1} a(k)a(n-1-k).
3: G.f.: A(x) = (1 - sqrt(1 - 4*x)) / (2*x), and satisfies A(x) = 1 + x*A(x)^2.

## Generating functions: What Sage can and cannot do.
What Sage easily does for us:
- Taking the recurrence
- Solving it, giving the generating function
- Expanding it; giving any (finite) number of terms

What Sage struggles with; it's something you can do, but it requires effort and knowledge:
- Giving a closed form for the coefficients
- Expanding infinite sums

Some other computer algebra systems do it better than SageMath. Some optional packages can be added to SageMath to do it, but it is still not that simple.

## Linear recurrence
The simpler case is a linear recurrence: something of the form $A_{n} = a\cdot a_{n-1}+b\cdot a_{n-2}+c\cdot a_{n-3}+\ldots$.

A linear recurrence is also called a C-finite sequence. Hence, we can explore the space of `CFiniteSequences`.

### Fibonacci numbers

Recall that the Fibonacci numbers are defined using $F_0 = 1$, $F_1 =1$ and $F_n = F_{n-1}+F_{n-2}$ for $n\geq 2$.

In [26]:
# Define the ring of C-finite sequences over the rationals; a C-finite sequence is a recurrence
C.<x> = CFiniteSequences(QQ)

# Give the recurrence to SageMath. This code can be modified for other recurrences.
fibo = C.from_recurrence([1,1],[1,1]) # Recurrence F_{n+2} = F_{n+1} + F_n with v_0 = v_1 = 1
fibo

C-finite sequence, generated by -1/(x^2 + x - 1)

We got a generating function for the sequence. Can we get other informations, like the coefficients or the partial fraction decomposition?

In [27]:
fibo.series(10).coefficients()

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [28]:
fibo[1000]

70330367711422815821835254877183549770181269836358732742604905087154537118196933579742249494562611733487750449241765991088186363265450223647106012053374121273867339111198139373125598767690091902245245323403501

We can even get easily the partial fraction decomposition...

In [29]:
# In class

In [30]:
# In class

In [31]:
ogf_fibo = fibo.ogf()  # ogf stands for 'ordinary generating function'
ogf_fibo

-1/(x^2 + x - 1)

In [32]:
ogf_fibo.partial_fraction_decomposition?

Here, however, the denominator cannot we factored over $\mathbb{Q}$, so it is already decomposed.

In [33]:
factor(x^2+x-1)

x^2 + x - 1

In [34]:
# In comparison...
factor(x^2-1)

(x - 1) * (x + 1)

There is even a way to 'guess' a generating function from a few coefficients. However, the sequence **must** be a linear recurrence.

In [35]:
C.guess([1,2,4,8,16,32])

C-finite sequence, generated by -1/2/(x - 1/2)

For this to work, we must feed the 'guess' function enough terms.

In [36]:
seq = C.from_recurrence([5,2,1],[7,9])
seq.series(10)

7 + 9*x + 23*x^2 + 76*x^3 + 167*x^4 + 434*x^5 + 1148*x^6 + 2851*x^7 + 7317*x^8 + 18759*x^9 + O(x^10)

In [37]:
# I secretly generated a sequence from a recurrence. Here are the first terms.
mystery = _.coefficients(); mystery

[7, 9, 23, 76, 167, 434, 1148, 2851, 7317, 18759]

In [38]:
C.guess([7, 9, 23, 76, 167, 434])

0

In [39]:
C.guess([7, 9, 23, 76, 167, 434, 1148, 2851, 7317])

C-finite sequence, generated by (-2/5*x - 7/5)/(x^3 + 2/5*x^2 + 1/5*x - 1/5)

Now that we have the sequence, we can ask for the recurrence. This is exactly what I inputed SageMath (in secret)!

In [40]:
_.recurrence_repr()

'homogeneous linear recurrence with constant coefficients of degree 3: a(n+3) = a(n+2) + 2*a(n+1) + 5*a(n), starting a(0...) = [7, 9, 23]'

# Takeaway

- SageMath knows a lot of tools to make our life with generating functions simpler.
- SageMath has a lot of things here and there for generating functions, but would benefit from a uniform framework and a greater ease of use!

**Open problem**: Make a uniform framework for generating functions in SageMath!